<!--<badge>--><a href="https://colab.research.google.com/github/Reversean/opt-labs/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Расчетное задание 2

## Марковские модели принятия решений

Выполнил студент гр. 3540901/21501 Лихолетов М.Д.

Руководитель, к.т.н., доц. Сиднев А.Г.

## Задача

Пример 5, стр. 272, из книги Макарова И.М. "Теория выбора и принятия решений".

Пусть имеется машина (станок), которая обслуживается периодически один раз в час. В каждый момент она может находиться в одном из двух состояний: рабочем (состояние 1) и нерабочем (состояние 2). Если машина на некотором шаге проработала непрерывно 1 час, то доход равен 3 рублям. При этом вероятность остаться на следующем шаге в состоянии 1 равна 0.7, а вероятность перейти в состояние 2 равна 0.3. Если машина отказала на некотором шаге, то ее можно отремонтировать двумя способами. Первый является ускоренным, требует затрат в 2 рубля (доход равен -2) и обеспечивает переход в состояние 1 с вероятностью 0.6. Второй, обычный способ требует затрат в 1 рубль и обеспечивает переход в состояние 1 с вероятностью 0.4.

Требуется найти оптимальную стратегию для $N = \infty$  методом итераций по стратегиям.

## Формирование параметров задачи

Машина может находиться в одном из двух состояниях:

- Рабочее ($S_1$)
- Нерабочее ($S_2$)

Для машины можно предпринимать следующие решения:

1. Ничего не делать ($X_1$)
2. Осуществить ускоренный ремонт ($X_2$)
3. Осуществить обычный ремонт ($X_3$)

Матрицы переходных вероятностей, соответствующие состояниям и решениям выше:

\begin{gather}
    P(i | X_{l_{i - 1}}) \equiv
    \begin{cases}
        P_1, \quad X_{l_{i - 1}} = X_1\\
        P_2, \quad X_{l_{i - 1}} = X_2\\
        P_3, \quad X_{l_{i - 1}} = X_3
    \end{cases}
\end{gather}

$$
P_1 =
\begin{bmatrix}
    0.7 & 0.3\\
    0 & 0
\end{bmatrix}
\quad
P_2 =
\begin{bmatrix}
    0 & 0\\
    0.6 & 0.4
\end{bmatrix}
\quad
P_3 =
\begin{bmatrix}
    0 & 0\\
    0.4 & 0.6
\end{bmatrix}
$$

где $P(i | X_{l_{i - 1}})$ – матрица переходов, применяемая на этапе $i$ на основе решения, принятого на этапе $i-1$, $X_{l_{i-1}}$ – решение, принятое на этапе $i$.

Матрицы доходов:

\begin{gather}
    R(i | X_{l_{i - 1}}) \equiv
    \begin{cases}
        R_1, \quad X_{l_{i - 1}} = X_1\\
        R_2, \quad X_{l_{i - 1}} = X_2\\
        R_3, \quad X_{l_{i - 1}} = X_3
    \end{cases}
\end{gather}

$$
R_1 =
\begin{bmatrix}
    3 & 3\\
    0 & 0
\end{bmatrix}
\quad
R_2 =
\begin{bmatrix}
    0 & 0\\
    -2 & -2
\end{bmatrix}
\quad
R_3 =
\begin{bmatrix}
    0 & 0\\
    -1 & -1
\end{bmatrix}
$$


где $R(i | X_{l_{i - 1}})$ – матрица доходов, применяемая на этапе $i$ на основе решения, принятого на этапе $i-1$, $X_{l_{i-1}}$ – решение, принятое на этапе $i$.

Требуется найти оптимальную стратегию $\tau$ – вектор решений, которые будут применены при соответствующих состояниях.

В текущих условиях могут быть предложены следующие стационарные стратегии:

- ускоренный ремонт в случае поломки ($\tau_1 = \{ X_1, X_2 \}$)
- обычный ремонт в случае поломки ($\tau_2 = \{ X_1, X_3 \}$)

Задачу поиска оптимальной стратегии на бесконечном горизонте событий методом итераций по стратегиям можно сформулировать как максимизацию ожидаемого дохода:

\begin{gather}
E + F(j) = \nu_j + \sum_{k = 1}^m{p_{jk} F(k)}\\
\nu_j = \sum_{k = 1}^m{p_{jk} r_{jk}}\\
j = \overline{1, m}\\
m = 2
\end{gather}

где $E$ – ожидаемый доход за один этап безотносительно к состоянию на следующем этапе, $F(j)$ – ожидаемый доход, если на следующем этапе будет осуществлен переход в состояние $S_j$, $\nu_j$ – ожидаемый доход относительно состояния на предыдущем этапе, если на следующем этапе будет осуществлен переход в состояние $S_j$, $p_{jk}$ – вероятность перехода из состояния $S_j$ в состояние $S_k$, $r_{jk}$ – доход при соответствующем переходе, $m$ – количество возможных состояний.

Итерационный процесс поиска решения состоит из 2 этапов:

1. Этап оценивания параметров
2. Этап улучшения стратегии

## 1 Итерация

### Этап оценивания параметров

На этапе оценивания требуется для рассматриваемой стратегии найти составляющие ожидаемого дохода ($E_{\tau}$ и $F_{\tau}(k)$), т.е. решить систему уравнений:

\begin{gather}
E_{\tau} + F_{\tau}(j) - \sum_{k=1}^{m} p_{jk}(\tau) F_{\tau}(k) = \nu_j(\tau)\\
\nu_j(\tau) = \sum_{k=1}^{m} p_{jk}(\tau) r_{jk}(\tau)\\
j = \overline{1, m}\\
F_{\tau}(m) = 0\\
m = 2
\end{gather}

где $\tau$ – произвольная стратегия.

Начать можно с решения системы уравнений для оценивания стратегии $\tau_1 = { X_1, X_2 }$ (ускоренный ремонт в случае поломки).

Матрицы вероятностей и доходов соответственно:

$$
P =
\begin{bmatrix}
    0.7 & 0.3\\
    0.6 & 0.4
\end{bmatrix}
\quad
R =
\begin{bmatrix}
    3 & 3\\
    -2 & -2
\end{bmatrix}
$$

Система уравнений для оценивания стратегии $\tau_1$:

\begin{cases}
    E_{\tau_1} + 0.3 F_{\tau_1}(1) = 3\\
    E_{\tau_1} - 0.6 F_{\tau_1}(1) = -2
\end{cases}

Функция для решения системы уравнений оценки стратегии:

In [1]:
from sympy import symbols, solve

E = symbols('E')
F = symbols('F')


def solve_eval(eq1, eq2):
    solving_result = solve([eq1, eq2], [E, F], dict=True)[0]
    result = dict((k, float(v)) for (k, v) in solving_result.items())
    print('E = {:.2f}'.format(result[E]))
    print('F = {:.2f}'.format(result[F]))
    return result

Решение системы уравнений для стратегии $\tau_1$:

In [2]:
eval1 = solve_eval(E + 0.3 * F - 3, E - 0.6 * F + 2)

E = 1.33
F = 5.56


### Этап улучшения стратегии

На этапе улучшения стратегии требуется для каждого состояния найти допустимое решение, при котором достигается максимальный суммарный ожидаемый доход с учетом значений ожидаемого дохода, определенных на этапе оценивания:

\begin{gather}
    \max_{X_i} \bigg( \nu_j(X_i) + \sum_{k=1}^m p_{jk}(X_i) F_{\tau}(k) \bigg)\\
    \nu_j(X_i) = \sum_{k=1}^{m} p_{jk}(X_i) r_{jk}(\tau)\\
    j = \overline{1, m}\\
    i = \overline{1, s}\\
    F_{\tau}(m) = 0\\
    m = 2\\
    s = 3
\end{gather}

Ожидаемые доходы для соответствующих состояний и решений при применении стратегии $\tau_1$:

In [3]:
from pandas import DataFrame

improve_table = DataFrame({
    'S': range(1, 3),
    'X1': [3 + 0.7 * eval1[F], 0],
    'X2': [0, -2 + 0.6 * eval1[F]],
    'X3': [0, -1 + 0.4 * eval1[F]]
})
improve_table.set_index('S')

,X1,X2,X3
S,,,
1,6.888889,0.000000,0.000000
2,0.000000,1.333333,1.222222


Оптимальным решением для состояния $S_1$ очевидно будет решение $X_1$, а для $S_2$ соответственно решение $X_2$.

Полученная стратегия полностью совпадает со стратегией $\tau_1$, а значит за одну итерацию удалось найти оптимальную стратегию.

## Вывод

В данной работе было рассмотрен метод итерации по стратегиям для решения марковской задачи принятия решения.

Т.к. по условиям имеется всего 2 адекватные стратегии, задача является довольно тривиальной, и удалось за одну итерацию найти оптимальную стратегию.

Стоит добавить, что если бы в первой итерации была бы выбрана стратегия $\{ X_1, X_3 \}$, то скорее всего на этапе улучшения стратегии полученная стратегия оказалась бы отличной от рассматриваемой, и пришлось бы проводить еще одну итерацию, рассмотренную в данном отчете.